In [299]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [300]:
client = MongoClient()

In [301]:
#Connecting to existing database and table within the database
db = client.images2
collection= db.image_data

In [302]:
print db
print collection

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'images2')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'images2'), u'image_data')


In [303]:
#Testing retreving data from Mongodb. Imp for when an img-no doesnt exist
cur = collection.find({'img_no':29},{'_id':0,'img_no':1,'prod_info_table':1})
cur.rewind()

try:
    print (cur)[0]
except:
    print "Not readable error"
    

{u'prod_info_table': {u'Item Shape': u'Round', u'Part Number': u'8932', u'Display Type': u'Analog', u'Metal stamp': u'316L', u'Case diameter': u'37 millimeters', u'Band width': u'15 millimeters', u'Dial color': u'Black', u'Model number': u'INVICTA-8932', u'Item weight': u'8.80 Ounces', u'Band Material': u'Stainless steel', u'Case material': u'Stainless steel', u'Band length': u'mens', u'Bezel material': u'Stainless steel', u'Special features': u'push-pull-crown, Luminous', u'Water resistant depth': u'660 Feet', u'Model Year': u'2014', u'Case Thickness': u'13 millimeters', u'Band Color': u'Silver', u'Brand, Seller, or Collection Name': u'Invicta'}, u'img_no': 29}


In [292]:
#total number of docs in collections
total_docs = collection.count()

In [338]:
#getting the data(img_no and prod_info_table) out of Mongodb
#5516 is the max img_no known through looking at data
prod_desc_table = []
img_no = []
for i in xrange(5940):
    cursor = collection.find({'img_no':i},{'_id':0,'img_no':1,'prod_info_table':1})
    #print cursor[0]
    cursor.rewind()
    try: 
        #We do cursor of 0 because cursor only has 1 result a dictionary. 
        #We are adding the img_no as another feature in the prod_info_table dictionary
        #This is to keep track of the image number for each row . Will be used in retirving the results when we know th
        #nearest neigbours for the model
        
        dict = cursor[0]["prod_info_table"]
        
        dict['Img_no'] = cursor[0]["img_no"]
        
        prod_desc_table.append(dict)
    
        
    except:
        pass # Some number might not exist in between due to broken links while scraping. 
        #This will ensure we move to the next image number without raising an error in that case

In [339]:
df = pd.DataFrame(prod_desc_table)

In [342]:
len(df.columns)

27

In [355]:
sum(df['Battery description'] == 'battery_type_lithium_metal')

1

In [350]:
df['Battery description'].unique() 

array([nan, u'battery_type_lithium_metal'], dtype=object)

In [356]:
df.columns

Index([                       u'Band Color',
                           u'Band Material',
                             u'Band length',
                              u'Band width',
                     u'Battery description',
                          u'Bezel material',
       u'Brand, Seller, or Collection Name',
                          u'Case Thickness',
                           u'Case diameter',
                           u'Case material',
                                   u'Clasp',
                              u'Dial color',
                            u'Display Type',
                                  u'Height',
                                  u'Img_no',
                             u'Item Length',
                              u'Item Shape',
                             u'Item weight',
                                  u'Length',
                                u'Material',
                             u'Metal stamp',
                              u'Model Year',
          

In [357]:
df.iloc[:,:10].head()

,Band Color,Band Material,Band length,Band width,Battery description,Bezel material,"Brand, Seller, or Collection Name",Case Thickness,Case diameter,Case material
0,Black,Resin,Men's Standard,22 millimeters,NaN,Resin,Casio,12 millimeters,43 millimeters,Resin
1,Black,Calfskin,Mens Standard,18 millimeters,NaN,Brass,Timex,8 millimeters,35 millimeters,Brass
2,Black,Resin,mens,18 millimeters,NaN,Plastic,Casio,8 millimeters,35 millimeters,Plastic
3,Black,Resin,mens,20 millimeters,NaN,Resin,Casio,7.8 millimeters,35 millimeters,Resin
4,Biege,Nylon,Mens-Standard,20 millimeters,NaN,Stainless Steel,Casio,11 millimeters,41 millimeters,Resin


In [359]:
df.iloc[:,10:20].head()

,Clasp,Dial color,Display Type,Height,Img_no,Item Length,Item Shape,Item weight,Length,Material
0,Buckle,Black,Analog,NaN,0,NaN,Round,2.08 Ounces,NaN,NaN
1,Buckle,White,Analog,NaN,1,NaN,Round,1.44 Ounces,NaN,NaN
2,Buckle,White,Analog,NaN,2,NaN,Round,18.14 Grams,NaN,NaN
3,Buckle,White,Analog,NaN,3,NaN,Round,NaN,NaN,NaN
4,Buckle,Beige,Analog,NaN,4,NaN,Round,16 Ounces,NaN,NaN


In [361]:
df.iloc[:,20:].head()

,Metal stamp,Model Year,Model number,Part Number,Special features,Water pressure resistance,Water resistant depth
0,NaN,2014,MRW200H-1BV,MRW200H-1BV,Water Resistant,NaN,330 Feet
1,NaN,2009,T2H281,T2H2819J,"measures-seconds, Light",NaN,99 Feet
2,NaN,2014,MQ24-7B2,MQ24-7B2,Water Resistant,NaN,NaN
3,NaN,NaN,MQ24-7B,MQ24-7B,Water Resistant,NaN,50
4,NaN,2014,FT-500WC-5BVCF,FT-500WC-5BVCF,Luminous,NaN,330 Feet


In [365]:
df.isnull().sum(axis =0)

Band Color                             83
Band Material                          33
Band length                           197
Band width                             72
Battery description                  4956
Bezel material                        331
Brand, Seller, or Collection Name       0
Case Thickness                        105
Case diameter                          29
Case material                         119
Clasp                                1768
Dial color                             21
Display Type                           72
Height                               4956
Img_no                                  0
Item Length                          4955
Item Shape                             73
Item weight                          2617
Length                               4956
Material                             4955
Metal stamp                          4460
Model Year                           1780
Model number                           34
Part Number                       

In [397]:
# Anything with more than  null values is dropped immediately 
df.drop(['Battery description','Water pressure resistance','Material','Height','Item Length','Metal stamp','Length','Part Number'],axis =1,inplace = True)

ValueError: labels ['Battery description' 'Water pressure resistance' 'Material' 'Height'
 'Item Length' 'Metal stamp' 'Length'] not contained in axis

In [369]:
df.shape

(4957, 20)

In [370]:
df.head()

,Band Color,Band Material,Band length,Band width,Bezel material,"Brand, Seller, or Collection Name",Case Thickness,Case diameter,Case material,Clasp,Dial color,Display Type,Img_no,Item Shape,Item weight,Model Year,Model number,Part Number,Special features,Water resistant depth
0,Black,Resin,Men's Standard,22 millimeters,Resin,Casio,12 millimeters,43 millimeters,Resin,Buckle,Black,Analog,0,Round,2.08 Ounces,2014,MRW200H-1BV,MRW200H-1BV,Water Resistant,330 Feet
1,Black,Calfskin,Mens Standard,18 millimeters,Brass,Timex,8 millimeters,35 millimeters,Brass,Buckle,White,Analog,1,Round,1.44 Ounces,2009,T2H281,T2H2819J,"measures-seconds, Light",99 Feet
2,Black,Resin,mens,18 millimeters,Plastic,Casio,8 millimeters,35 millimeters,Plastic,Buckle,White,Analog,2,Round,18.14 Grams,2014,MQ24-7B2,MQ24-7B2,Water Resistant,NaN
3,Black,Resin,mens,20 millimeters,Resin,Casio,7.8 millimeters,35 millimeters,Resin,Buckle,White,Analog,3,Round,NaN,NaN,MQ24-7B,MQ24-7B,Water Resistant,50
4,Biege,Nylon,Mens-Standard,20 millimeters,Stainless Steel,Casio,11 millimeters,41 millimeters,Resin,Buckle,Beige,Analog,4,Round,16 Ounces,2014,FT-500WC-5BVCF,FT-500WC-5BVCF,Luminous,330 Feet


In [409]:
#Remove part number
df.drop(['Part Number'],axis =1,inplace = True)

In [411]:
print df.columns
df.shape

Index([                       u'Band Color',
                           u'Band Material',
                             u'Band length',
                              u'Band width',
                          u'Bezel material',
       u'Brand, Seller, or Collection Name',
                          u'Case Thickness',
                           u'Case diameter',
                           u'Case material',
                                   u'Clasp',
                              u'Dial color',
                            u'Display Type',
                                  u'Img_no',
                              u'Item Shape',
                             u'Item weight',
                              u'Model Year',
                            u'Model number',
                        u'Special features',
                   u'Water resistant depth'],
      dtype='object')


(4957, 19)

In [378]:
df['Clasp'].unique() #Length is 118

array([u'Buckle', nan, u'Hook-and-loop', u'fold-over-push-button-clasp',
       u'Deployant-clasp', u'deployant-clasp',
       u'Fold Over Push Button Clasp With Safety', u'Expansion-band',
       u'double-locking-fold-over', u'single-locking-fold-over',
       u'deployant-clasp-with-push-button',
       u'hidden-push-button-fold-over-clasp',
       u'Fold-over-clasp-with-push-button',
       u'deployant-push-button-clasp-with-buckle', u'Hook Buckle',
       u'deployant-buckle', u'Deployant-clasp-with-push-button',
       u'Fold-over-clasp-with-hidden-double-push-button',
       u'buckle closure', u'Push button deployment',
       u'fold-over-clasp-with-double-push-button', u'Deployment',
       u'Foldover with Safety', u'Pin Buckle', u'fold-over',
       u'Fold-over Clasp', u'SIZING CLASP SYSTEM',
       u'Fold Over Clasp with Safety', u'Fold Over Clasp With Safety',
       u'hook-clasp', u'push-button-fold-over-clasp',
       u'Deployment-buckle-with-push-button',
       u'fold-over-

In [170]:
len_max_ind = 0 # index of element with Max Length of table
len_max = 0 # max length
ind = [] # All the indexes with max length to compare if table has same c
# Getting the table with max components to be used to derive the table contents. The one which has the max

for i,result in enumerate(results):
    if len(result[1]) > len_max :
        len_max = len(result[1])
        len_max_index = i
        
    elif  (len(result[1]) == len_max):
        ind.append((i,len_max))
        
    else:
        pass

In [178]:
print len_max_index,len_max
print ind

1213 25
[(1, 23), (3, 23), (5, 23), (14, 24), (31, 24), (43, 24), (44, 24), (63, 24), (68, 24), (69, 24), (81, 24), (85, 24), (89, 24), (94, 24), (108, 24), (115, 24), (119, 24), (122, 24), (152, 24), (168, 24), (179, 24), (193, 24), (199, 24), (254, 24), (270, 24), (272, 24), (284, 24), (302, 24), (314, 24), (343, 24), (360, 24), (401, 24), (408, 24), (412, 24), (420, 24), (428, 24), (435, 24), (439, 24), (457, 24), (485, 24), (503, 24), (510, 24), (546, 24), (550, 24), (574, 24), (619, 24), (621, 24), (635, 24), (658, 24), (661, 24), (695, 24), (699, 24), (727, 24), (737, 24), (743, 24), (744, 24), (748, 24), (828, 24), (834, 24), (845, 24), (862, 24), (863, 24), (872, 24), (874, 24), (905, 24), (906, 24), (911, 24), (917, 24), (959, 24), (963, 24), (973, 24), (974, 24), (1036, 24), (1071, 24), (1078, 24), (1087, 24), (1088, 24), (1093, 24), (1127, 24), (1134, 24), (1137, 24), (1142, 24), (1156, 24), (1157, 24), (1158, 24), (1159, 24), (1181, 24), (1206, 24), (1212, 24)]


We can see above that  index 1213 has the largest number of table contents we'll focus on that to get the 
table contents

In [224]:
#Extracting the Column names
prod_table = (results[1213][1])
products = []
column_names = []
for i in prod_table:
    column_names.append(str(i.rsplit(', u')[0]))
    products.append(i.rsplit(', u')[1])
column_names

["(u'Brand, Seller, or Collection Name'",
 "(u'Model number'",
 "(u'Part Number'",
 "(u'Model Year'",
 "(u'Item Shape'",
 "(u'Dial window material type\\ufeff'",
 "(u'Display Type'",
 "(u'Clasp'",
 "(u'Metal stamp'",
 "(u'Case material'",
 "(u'Case diameter'",
 "(u'Case Thickness'",
 "(u'Band Material'",
 "(u'Band length'",
 "(u'Band width'",
 "(u'Band Color'",
 "(u'Dial color'",
 "(u'Bezel material'",
 "(u'Bezel function\\ufeff'",
 "(u'Calendar\\ufeff'",
 "(u'Special features'",
 "(u'Item weight'",
 "(u'Movement\\ufeff'",
 "(u'Water pressure resistance'",
 "(u'Water resistant depth'"]

column_names

In [221]:
columns = []
for col in column_names:
    columns.append(((col.replace("(","")).replace("u'","")).replace("'",""))

In [222]:
columns

['Brand, Seller, or Collection Name',
 'Model number',
 'Part Number',
 'Model Year',
 'Item Shape',
 'Dial window material type\\ufeff',
 'Display Type',
 'Clasp',
 'Metal stamp',
 'Case material',
 'Case diameter',
 'Case Thickness',
 'Band Material',
 'Band length',
 'Band width',
 'Band Color',
 'Dial color',
 'Bezel material',
 'Bezel function\\ufeff',
 'Calendar\\ufeff',
 'Special features',
 'Item weight',
 'Movement\\ufeff',
 'Water pressure resistance',
 'Water resistant depth']

In [225]:
products

[u"'Citizen')",
 u"'AT4004-52E')",
 u"'AT4004-52E')",
 u"'2011')",
 u"'Round')",
 u"'Anti reflective sapphire')",
 u"'Analog')",
 u"'Fold-Over Clasp with Hidden Double Push-Button\\ufeff')",
 u"'None')",
 u"'Stainless steel')",
 u"'42 millimeters')",
 u"'13 millimeters')",
 u"'Two-tone stainless steel')",
 u'"Men\'s Standard")',
 u"'24 millimeters')",
 u"'Silver')",
 u"'Black')",
 u"'Stainless steel')",
 u"'Stationary')",
 u"'Perpetual calendar')",
 u"'Radio controlled, Chronograph, Luminous, Stop watch, Tachometer')",
 u"'6.24 Ounces')",
 u"'Japanese quartz')",
 u"'20 bar')",
 u"'660 Feet')"]

In [315]:
pd.DataFrame(results[1][1])

ValueError: If using all scalar values, you must pass an index

In [231]:
import json
for i in  prod_table:
    json.loads(i)

ValueError: No JSON object could be decoded

In [247]:
db2 = client.images2
col2 = db2.image_data



In [248]:
x = col2.find_one()['prod_info_table']

In [249]:
for i in x:
    print i.keys()
    

[u'Brand, Seller, or Collection Name']
[u'Model number']
[u'Part Number']
[u'Model Year']
[u'Item Shape']
[u'Display Type']
[u'Clasp']
[u'Case material']
[u'Case diameter']
[u'Case Thickness']
[u'Band Material']
[u'Band length']
[u'Band width']
[u'Band Color']
[u'Dial color']
[u'Bezel material']
[u'Special features']
[u'Item weight']
[u'Water resistant depth']
